In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from torch.optim import Adam
from torchsummary import summary  # Import the summary function
from PyTorchLabFlow import test_mods

In [2]:
class ResN50_40(nn.Module):
    def __init__(self):
        super(ResN50_40, self).__init__()

        self.desc = "[3,224,224] -> ResNet50preTrained(-40:)[2048] -> 256 -> 74"

        # Step 1: Load ResNet50 base model (without pre-trained weights)
        self.base_model = models.resnet50(weights=None)  # No pretrained weights at first
        self.base_model.fc = nn.Identity()  # Remove the final fully connected (fc) layer

        # Step 2: Load the pre-trained weights into the base model
        state_dict = torch.load("preTrained/resnet50_pretrained.pth", weights_only=True)
        self.base_model.load_state_dict(state_dict, strict=False)  # Load the pre-trained weights

        # Custom layers after the base model
        self.fc1 = nn.Linear(2048, 256)  # Fully connected layer
        self.batch_norm = nn.BatchNorm1d(256)  # Apply BatchNorm after fc1
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 74)  # Output layer (num_classes will be 74)

        # Step 3: Create a list of all parameters
        self.params = list(self.base_model.named_parameters())
        # print(f"Total parameters: {len(self.params)}")

        # Identify the last 10 layers
        self.last_40_params = self.params[-40:]  # Get the last 10 parameters

        # Step 4: Freeze all layers except the last 10 layers
        for name, param in self.base_model.named_parameters():
            param.requires_grad = False  # Freeze all layers initially

        # Step 5: Unfreeze the last 10 layers
        for name, param in self.last_40_params:
            param.requires_grad = True  # Unfreeze the last 10 layers

    def forward(self, x):
        # Forward pass through ResNet50 base model
        x = self.base_model(x)  # Get features from ResNet50
        x = self.fc1(x)          # Apply fully connected layer 1
        x = self.batch_norm(x)   # Apply batch normalization after fc1
        x = self.relu(x)         # ReLU activation
        x = self.dropout(x)      # Apply dropout
        x = self.fc2(x)          # Output layer
        return x

# Initialize the model
model = ResN50_40()

# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)

# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for ResNet50


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [2]:
class ResN50_un(nn.Module):
    def __init__(self):
        super(ResN50_un, self).__init__()

        self.desc = "[3,224,224] -> ResNet50[2048] -> 256 -> 74"

        # Step 1: Load ResNet50 base model (without pre-trained weights)
        self.base_model = models.resnet50(weights=None)  # No pretrained weights at first
        self.base_model.fc = nn.Identity()  # Remove the final fully connected (fc) layer

        # # Step 2: Load the pre-trained weights into the base model
        # state_dict = torch.load("preTrained/resnet50_pretrained.pth", weights_only=True)
        # self.base_model.load_state_dict(state_dict, strict=False)  # Load the pre-trained weights

        # Custom layers after the base model
        self.fc1 = nn.Linear(2048, 256)  # Fully connected layer
        self.batch_norm = nn.BatchNorm1d(256)  # Apply BatchNorm after fc1
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 74)  # Output layer (num_classes will be 74)

        # Step 3: Create a list of all parameters
        # self.params = list(self.base_model.named_parameters())
        # print(f"Total parameters: {len(self.params)}")

        # Identify the last 10 layers
        # self.last_35_params = self.params[-35:]  # Get the last 10 parameters

        # Step 4: Freeze all layers except the last 10 layers
        # for name, param in self.base_model.named_parameters():
        #     param.requires_grad = False  # Freeze all layers initially

        # # Step 5: Unfreeze the last 10 layers
        # for name, param in self.last_35_params:
        #     param.requires_grad = True  # Unfreeze the last 10 layers

    def forward(self, x):
        # Forward pass through ResNet50 base model
        x = self.base_model(x)  # Get features from ResNet50
        x = self.fc1(x)          # Apply fully connected layer 1
        x = self.batch_norm(x)   # Apply batch normalization after fc1
        x = self.relu(x)         # ReLU activation
        x = self.dropout(x)      # Apply dropout
        x = self.fc2(x)          # Output layer
        return x

# Initialize the model
model = ResN50_un()

# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)

# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for ResNet50


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [3]:
class ResN50_35(nn.Module):
    def __init__(self):
        super(ResN50_35, self).__init__()

        self.desc = "[3,224,224] -> ResNet50preTrained(-35:)[2048] -> 256 -> 74"

        # Step 1: Load ResNet50 base model (without pre-trained weights)
        self.base_model = models.resnet50(weights=None)  # No pretrained weights at first
        self.base_model.fc = nn.Identity()  # Remove the final fully connected (fc) layer

        # Step 2: Load the pre-trained weights into the base model
        state_dict = torch.load("preTrained/resnet50_pretrained.pth", weights_only=True)
        self.base_model.load_state_dict(state_dict, strict=False)  # Load the pre-trained weights

        # Custom layers after the base model
        self.fc1 = nn.Linear(2048, 256)  # Fully connected layer
        self.batch_norm = nn.BatchNorm1d(256)  # Apply BatchNorm after fc1
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 74)  # Output layer (num_classes will be 74)

        # Step 3: Create a list of all parameters
        self.params = list(self.base_model.named_parameters())
        # print(f"Total parameters: {len(self.params)}")

        # Identify the last 10 layers
        self.last_35_params = self.params[-35:]  # Get the last 10 parameters

        # Step 4: Freeze all layers except the last 10 layers
        for name, param in self.base_model.named_parameters():
            param.requires_grad = False  # Freeze all layers initially

        # Step 5: Unfreeze the last 10 layers
        for name, param in self.last_35_params:
            param.requires_grad = True  # Unfreeze the last 10 layers

    def forward(self, x):
        # Forward pass through ResNet50 base model
        x = self.base_model(x)  # Get features from ResNet50
        x = self.fc1(x)          # Apply fully connected layer 1
        x = self.batch_norm(x)   # Apply batch normalization after fc1
        x = self.relu(x)         # ReLU activation
        x = self.dropout(x)      # Apply dropout
        x = self.fc2(x)          # Output layer
        return x

# Initialize the model
model = ResN50_35()

# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)

# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for ResNet50


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [2]:
class ResN50_25(nn.Module):
    def __init__(self):
        super(ResN50_25, self).__init__()

        self.desc = "[3,224,224] -> ResNet50preTrained(-15:)[2048] -> 256 -> 74"

        # Step 1: Load ResNet50 base model (without pre-trained weights)
        self.base_model = models.resnet50(weights=None)  # No pretrained weights at first
        self.base_model.fc = nn.Identity()  # Remove the final fully connected (fc) layer

        # Step 2: Load the pre-trained weights into the base model
        state_dict = torch.load("preTrained/resnet50_pretrained.pth", weights_only=True)
        self.base_model.load_state_dict(state_dict, strict=False)  # Load the pre-trained weights

        # Custom layers after the base model
        self.fc1 = nn.Linear(2048, 256)  # Fully connected layer
        self.batch_norm = nn.BatchNorm1d(256)  # Apply BatchNorm after fc1
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 74)  # Output layer (num_classes will be 74)

        # Step 3: Create a list of all parameters
        self.params = list(self.base_model.named_parameters())
        # print(f"Total parameters: {len(self.params)}")

        # Identify the last 10 layers
        self.last_25_params = self.params[-25:]  # Get the last 10 parameters

        # Step 4: Freeze all layers except the last 10 layers
        for name, param in self.base_model.named_parameters():
            param.requires_grad = False  # Freeze all layers initially

        # Step 5: Unfreeze the last 10 layers
        for name, param in self.last_25_params:
            param.requires_grad = True  # Unfreeze the last 10 layers

    def forward(self, x):
        # Forward pass through ResNet50 base model
        x = self.base_model(x)  # Get features from ResNet50
        x = self.fc1(x)          # Apply fully connected layer 1
        x = self.batch_norm(x)   # Apply batch normalization after fc1
        x = self.relu(x)         # ReLU activation
        x = self.dropout(x)      # Apply dropout
        x = self.fc2(x)          # Output layer
        return x

# Initialize the model
model = ResN50_25()

# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)

# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for ResNet50


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [2]:
class ResN50_20(nn.Module):
    def __init__(self):
        super(ResN50_20, self).__init__()

        self.desc = "[3,224,224] -> ResNet50preTrained(-15:)[2048] -> 256 -> 74"

        # Step 1: Load ResNet50 base model (without pre-trained weights)
        self.base_model = models.resnet50(weights=None)  # No pretrained weights at first
        self.base_model.fc = nn.Identity()  # Remove the final fully connected (fc) layer

        # Step 2: Load the pre-trained weights into the base model
        state_dict = torch.load("preTrained/resnet50_pretrained.pth", weights_only=True)
        self.base_model.load_state_dict(state_dict, strict=False)  # Load the pre-trained weights

        # Custom layers after the base model
        self.fc1 = nn.Linear(2048, 256)  # Fully connected layer
        self.batch_norm = nn.BatchNorm1d(256)  # Apply BatchNorm after fc1
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 74)  # Output layer (num_classes will be 74)

        # Step 3: Create a list of all parameters
        self.params = list(self.base_model.named_parameters())
        # print(f"Total parameters: {len(self.params)}")

        # Identify the last 10 layers
        self.last_15_params = self.params[-20:]  # Get the last 10 parameters

        # Step 4: Freeze all layers except the last 10 layers
        for name, param in self.base_model.named_parameters():
            param.requires_grad = False  # Freeze all layers initially

        # Step 5: Unfreeze the last 10 layers
        for name, param in self.last_15_params:
            param.requires_grad = True  # Unfreeze the last 10 layers

    def forward(self, x):
        # Forward pass through ResNet50 base model
        x = self.base_model(x)  # Get features from ResNet50
        x = self.fc1(x)          # Apply fully connected layer 1
        x = self.batch_norm(x)   # Apply batch normalization after fc1
        x = self.relu(x)         # ReLU activation
        x = self.dropout(x)      # Apply dropout
        x = self.fc2(x)          # Output layer
        return x

# Initialize the model
model = ResN50_20()

# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)

# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for ResNet50


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [18]:

class Vgg19_5(nn.Module):
    def __init__(self):
        super(Vgg19_5, self).__init__()

        self.desc = "[3,224,224] -> VGG19preTrained(-15:)[4096] -> 256 -> 74"

        # Step 1: Load VGG19 base model (without pre-trained weights)
        self.base_model = models.vgg19(weights=None)  # Load VGG19 without pre-trained weights
        self.base_model.classifier = nn.Sequential(*list(self.base_model.classifier.children())[:-1])  # Remove the final fully connected (fc) layer

        # Step 2: Load the pre-trained weights into the base model
        state_dict = torch.load("preTrained/vgg19_pretrained.pth", weights_only=True)  # Path to custom pretrained weights
        self.base_model.load_state_dict(state_dict, strict=False)  # Load the state_dict, assuming you have custom weights

        # Custom layers after the base model
        self.fc1 = nn.Linear(4096, 256)  # Custom fully connected layer for 4096 input features (VGG19 FC layer before output)
        self.batch_norm = nn.BatchNorm1d(256)  # Apply BatchNorm after fc1
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 74)  # Output layer (num_classes will be 74)

        # Step 3: Create a list of all parameters
        self.params = list(self.base_model.named_parameters())
        print(f"Total parameters: {len(self.params)}")

        # Identify the last 10 layers
        self.last_5_params = self.params[-2:]  # Get the last 10 parameters

        # Step 4: Freeze all layers except the last 10 layers
        for name, param in self.base_model.named_parameters():
            param.requires_grad = False  # Freeze all layers initially
        # Step 5: Unfreeze the last 10 layers
        for name, param in self.last_5_params:
            param.requires_grad = True  # Unfreeze the last 10 layers

    def forward(self, x):
        # Forward pass through VGG19 base model
        x = self.base_model(x)  # Get features from VGG19
        x = self.fc1(x)         # Apply fully connected layer 1
        x = self.batch_norm(x)  # Apply batch normalization after fc1
        x = self.relu(x)        # ReLU activation
        x = self.dropout(x)     # Apply dropout
        x = self.fc2(x)         # Output layer
        return x

# Initialize model
model = Vgg19_5()

# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)

# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for VGG19


Total parameters: 36
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-

In [5]:
class ResN50_15(nn.Module):
    def __init__(self):
        super(ResN50_15, self).__init__()

        self.desc = "[3,224,224] -> ResNet50preTrained(-15:)[2048] -> 256 -> 74"

        # Step 1: Load ResNet50 base model (without pre-trained weights)
        self.base_model = models.resnet50(weights=None)  # No pretrained weights at first
        self.base_model.fc = nn.Identity()  # Remove the final fully connected (fc) layer

        # Step 2: Load the pre-trained weights into the base model
        state_dict = torch.load("preTrained/resnet50_pretrained.pth", weights_only=True)
        self.base_model.load_state_dict(state_dict, strict=False)  # Load the pre-trained weights

        # Custom layers after the base model
        self.fc1 = nn.Linear(2048, 256)  # Fully connected layer
        self.batch_norm = nn.BatchNorm1d(256)  # Apply BatchNorm after fc1
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 74)  # Output layer (num_classes will be 74)

        # Step 3: Create a list of all parameters
        self.params = list(self.base_model.named_parameters())
        # print(f"Total parameters: {len(self.params)}")

        # Identify the last 10 layers
        self.last_15_params = self.params[-15:]  # Get the last 10 parameters

        # Step 4: Freeze all layers except the last 10 layers
        for name, param in self.base_model.named_parameters():
            param.requires_grad = False  # Freeze all layers initially

        # Step 5: Unfreeze the last 10 layers
        for name, param in self.last_15_params:
            param.requires_grad = True  # Unfreeze the last 10 layers

    def forward(self, x):
        # Forward pass through ResNet50 base model
        x = self.base_model(x)  # Get features from ResNet50
        x = self.fc1(x)          # Apply fully connected layer 1
        x = self.batch_norm(x)   # Apply batch normalization after fc1
        x = self.relu(x)         # ReLU activation
        x = self.dropout(x)      # Apply dropout
        x = self.fc2(x)          # Output layer
        return x

# Initialize the model
model = ResN50_15()

# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)

# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for ResNet50


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [36]:

class EffiB3_10(nn.Module):
    def __init__(self):
        super(EffiB3_10, self).__init__()
        self.desc = "[3,224,224] ->EfficientNetB3preTrained(-10:)[1536] ->256 ->74"
        # Step 1: Load EfficientNet-B3 base model (without pre-trained weights)
        self.base_model = models.efficientnet_b3(weights=None)  # No pretrained weights at first
        self.base_model.classifier = nn.Identity()  # Remove final classification layer
        
        # Step 2: Load the pre-trained weights into the base model
        state_dict = torch.load("preTrained/efficientnet_b3_weights.pth", weights_only=True)
        # Load the filtered state_dict into the model
        self.base_model.load_state_dict(state_dict,strict=False)
        # print("EfficientNet-B3 weights loaded successfully.")
        
        # Custom layers added after the base model
        self.batch_norm = nn.BatchNorm1d(1536)  # Output size of EfficientNet-B3 before classifier is 1536
        self.fc1 = nn.Linear(1536, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 74)
        
         # Step 3: Create a list of all parameters
        self.params = list(self.base_model.named_parameters())
        # print(f"Total parameters: {len(self.params)}")

        # Identify the last 10 layers
        self.last_10_params = self.params[-10:]  # Get the last 10 parameters

        # Step 4: Freeze all layers except the last 10 layers
        for name, param in self.base_model.named_parameters():
            param.requires_grad = False  # Freeze all layers initially

        # Step 5: Unfreeze the last 10 layers
        for name, param in self.last_10_params:
            param.requires_grad = True  # Unfreeze the last 10 layers
        
    def forward(self, x):
        # Forward pass through EfficientNet-B3 base model
        x = self.base_model(x)  # Get features from EfficientNet-B3
        x = self.batch_norm(x)   # Apply batch normalization
        x = self.fc1(x)          # Apply fully connected layer 1
        x = self.relu(x)         # ReLU activation
        x = self.dropout(x)      # Apply dropout
        x = self.fc2(x)          # Output layer
        return x

# Initialize the model (freeze base model layers by default)
model = EffiB3_10()
# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)
# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for EfficientNet-B3


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 40, 112, 112]           1,080
       BatchNorm2d-2         [-1, 40, 112, 112]              80
              SiLU-3         [-1, 40, 112, 112]               0
            Conv2d-4         [-1, 40, 112, 112]             360
       BatchNorm2d-5         [-1, 40, 112, 112]              80
              SiLU-6         [-1, 40, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 40, 1, 1]               0
            Conv2d-8             [-1, 10, 1, 1]             410
              SiLU-9             [-1, 10, 1, 1]               0
           Conv2d-10             [-1, 40, 1, 1]             440
          Sigmoid-11             [-1, 40, 1, 1]               0
SqueezeExcitation-12         [-1, 40, 112, 112]               0
           Conv2d-13         [-1, 24, 112, 112]             960
      BatchNorm2d-14         [-1, 24, 1

In [32]:
class ResN50_10(nn.Module):
    def __init__(self):
        super(ResN50_10, self).__init__()

        self.desc = "[3,224,224] -> ResNet50preTrained[2048] -> 256 -> 74"

        # Step 1: Load ResNet50 base model (without pre-trained weights)
        self.base_model = models.resnet50(weights=None)  # No pretrained weights at first
        self.base_model.fc = nn.Identity()  # Remove the final fully connected (fc) layer

        # Step 2: Load the pre-trained weights into the base model
        state_dict = torch.load("preTrained/resnet50_pretrained.pth", weights_only=True)
        self.base_model.load_state_dict(state_dict, strict=False)  # Load the pre-trained weights

        # Custom layers after the base model
        self.fc1 = nn.Linear(2048, 256)  # Fully connected layer
        self.batch_norm = nn.BatchNorm1d(256)  # Apply BatchNorm after fc1
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 74)  # Output layer (num_classes will be 74)

        # Step 3: Create a list of all parameters
        self.params = list(self.base_model.named_parameters())
        # print(f"Total parameters: {len(self.params)}")

        # Identify the last 10 layers
        self.last_10_params = self.params[-10:]  # Get the last 10 parameters

        # Step 4: Freeze all layers except the last 10 layers
        for name, param in self.base_model.named_parameters():
            param.requires_grad = False  # Freeze all layers initially

        # Step 5: Unfreeze the last 10 layers
        for name, param in self.last_10_params:
            param.requires_grad = True  # Unfreeze the last 10 layers

    def forward(self, x):
        # Forward pass through ResNet50 base model
        x = self.base_model(x)  # Get features from ResNet50
        x = self.fc1(x)          # Apply fully connected layer 1
        x = self.batch_norm(x)   # Apply batch normalization after fc1
        x = self.relu(x)         # ReLU activation
        x = self.dropout(x)      # Apply dropout
        x = self.fc2(x)          # Output layer
        return x

# Initialize the model
model = ResN50_10()

# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)

# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for ResNet50


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [29]:
df = [name  for name,param  in model.base_model.named_parameters() ]
# len(list(df.keys()))
df,len(df)

(['conv1.weight',
  'bn1.weight',
  'bn1.bias',
  'layer1.0.conv1.weight',
  'layer1.0.bn1.weight',
  'layer1.0.bn1.bias',
  'layer1.0.conv2.weight',
  'layer1.0.bn2.weight',
  'layer1.0.bn2.bias',
  'layer1.0.conv3.weight',
  'layer1.0.bn3.weight',
  'layer1.0.bn3.bias',
  'layer1.0.downsample.0.weight',
  'layer1.0.downsample.1.weight',
  'layer1.0.downsample.1.bias',
  'layer1.1.conv1.weight',
  'layer1.1.bn1.weight',
  'layer1.1.bn1.bias',
  'layer1.1.conv2.weight',
  'layer1.1.bn2.weight',
  'layer1.1.bn2.bias',
  'layer1.1.conv3.weight',
  'layer1.1.bn3.weight',
  'layer1.1.bn3.bias',
  'layer1.2.conv1.weight',
  'layer1.2.bn1.weight',
  'layer1.2.bn1.bias',
  'layer1.2.conv2.weight',
  'layer1.2.bn2.weight',
  'layer1.2.bn2.bias',
  'layer1.2.conv3.weight',
  'layer1.2.bn3.weight',
  'layer1.2.bn3.bias',
  'layer2.0.conv1.weight',
  'layer2.0.bn1.weight',
  'layer2.0.bn1.bias',
  'layer2.0.conv2.weight',
  'layer2.0.bn2.weight',
  'layer2.0.bn2.bias',
  'layer2.0.conv3.weight',


In [ ]:
len(list(model.base_model.layer4.children()))

In [2]:

class ResN50(nn.Module):
    def __init__(self):
        super(ResN50, self).__init__()

        self.desc = "[3,224,224] -> ResNet50preTrained[2048] -> 256 -> 74"

        # Step 1: Load ResNet50 base model (without pre-trained weights)
        self.base_model = models.resnet50(weights=None)  # No pretrained weights at first
        self.base_model.fc = nn.Identity()  # Remove the final fully connected (fc) layer

        # Step 2: Load the pre-trained weights into the base model
        state_dict = torch.load("preTrained/resnet50_pretrained.pth", weights_only=True)
        self.base_model.load_state_dict(state_dict, strict=False)  # Uncomment if you have custom pretrained weights

        # Custom layers after the base model
        self.fc1 = nn.Linear(2048, 256)  # Updated: Custom fully connected layer for 2048 input features
        self.batch_norm = nn.BatchNorm1d(256)  # Apply BatchNorm after fc1
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 74)  # Output layer (num_classes will be 74)

        # Step 3: Freeze the base model layers if specified
        for param in self.base_model.parameters():
                param.requires_grad = False  # Freeze base model layers
        
    def forward(self, x):
        # Forward pass through ResNet50 base model
        x = self.base_model(x)  # Get features from ResNet50
        x = self.fc1(x)          # Apply fully connected layer 1
        x = self.batch_norm(x)   # Apply batch normalization after fc1
        x = self.relu(x)         # ReLU activation
        x = self.dropout(x)      # Apply dropout
        x = self.fc2(x)          # Output layer
        return x




model = ResN50()
# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)
# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for EfficientNet-B3

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [10]:


class EfficientNetB3Model(nn.Module):
    def __init__(self):
        super(EfficientNetB3Model, self).__init__()
        
        # Step 1: Load EfficientNet-B3 base model (without pre-trained weights)
        self.base_model = models.efficientnet_b3(weights=None)  # No pretrained weights at first
        self.base_model.classifier = nn.Identity()  # Remove final classification layer
        
        # Step 2: Load the pre-trained weights into the base model
        state_dict = torch.load("preTrained/efficientnet_b3_weights.pth", weights_only=True)
        # Remove classifier weights from state_dict to avoid the mismatch
        state_dict = {k: v for k, v in state_dict.items() if 'classifier' not in k}
        
        # Load the filtered state_dict into the model
        self.base_model.load_state_dict(state_dict)
        # print("EfficientNet-B3 weights loaded successfully.")
        
        # Custom layers added after the base model
        self.batch_norm = nn.BatchNorm1d(1536)  # Output size of EfficientNet-B3 before classifier is 1536
        self.fc1 = nn.Linear(1536, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 74)
        
        # Step 3: Freeze the base model layers if specified
        for param in self.base_model.parameters():
                param.requires_grad = False  # Freeze base model layers
        
    def forward(self, x):
        # Forward pass through EfficientNet-B3 base model
        x = self.base_model(x)  # Get features from EfficientNet-B3
        x = self.batch_norm(x)   # Apply batch normalization
        x = self.fc1(x)          # Apply fully connected layer 1
        x = self.relu(x)         # ReLU activation
        x = self.dropout(x)      # Apply dropout
        x = self.fc2(x)          # Output layer
        return x

# Initialize the model (freeze base model layers by default)
model = EfficientNetB3Model()
# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)
# Display model summary
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for EfficientNet-B3


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 40, 112, 112]           1,080
       BatchNorm2d-2         [-1, 40, 112, 112]              80
              SiLU-3         [-1, 40, 112, 112]               0
            Conv2d-4         [-1, 40, 112, 112]             360
       BatchNorm2d-5         [-1, 40, 112, 112]              80
              SiLU-6         [-1, 40, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 40, 1, 1]               0
            Conv2d-8             [-1, 10, 1, 1]             410
              SiLU-9             [-1, 10, 1, 1]               0
           Conv2d-10             [-1, 40, 1, 1]             440
          Sigmoid-11             [-1, 40, 1, 1]               0
SqueezeExcitation-12         [-1, 40, 112, 112]               0
           Conv2d-13         [-1, 24, 112, 112]             960
      BatchNorm2d-14         [-1, 24, 1

In [8]:
import torch
import torch.nn as nn
from torchvision import models

class EfficientNetB3Model(nn.Module):
    def __init__(self):
        super(EfficientNetB3Model, self).__init__()
        
        # Load EfficientNet-B3 base model (without pre-trained weights)
        self.base_model = models.efficientnet_b3(weights=None)  # No pretrained weights at first
        self.base_model.classifier = nn.Identity()  # Remove final classification layer
        
        # Load pre-trained weights
        state_dict = torch.load("preTrained/efficientnet_b3_weights.pth", weights_only=True)
        # Remove classifier weights from state_dict to avoid mismatch
        state_dict = {k: v for k, v in state_dict.items() if 'classifier' not in k}
        
        # Load filtered state_dict into the model
        self.base_model.load_state_dict(state_dict)
        
        # Custom layers added after the base model
        self.batch_norm = nn.BatchNorm1d(1536)  # Output size of EfficientNet-B3 before classifier is 1536
        self.fc1 = nn.Linear(1536, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 74)
        
        # Freeze base model layers if specified
        for param in self.base_model.parameters():
                param.requires_grad = False  # Freeze base model layers

        # Enable mixed-precision training if needed (requires PyTorch AMP)
        
    def forward(self, x):
        # Automatic Mixed Precision (AMP) context manager for lower precision computations
        with torch.amp.autocast(device_type='cuda'):  # Updated usage for AMP
                x = self.base_model(x)  # Get features from EfficientNet-B3
                x = self.batch_norm(x)   # Apply batch normalization
                x = self.fc1(x)          # Apply fully connected layer 1
                x = self.relu(x)         # ReLU activation
                x = self.dropout(x)      # Apply dropout
                x = self.fc2(x)          # Output layer
        return x

# Initialize the model with the option to use AMP (mixed precision)
use_amp = True  # Set to True if you want to use mixed precision
model = EfficientNetB3Model()

# Assuming 'device' is the variable where the device is specified, e.g., 'cuda' or 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU if available, otherwise CPU)
model = model.to(device)
summary(model, input_size=(3, 224, 224))  # Input size (C, H, W) for EfficientNet-B3


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 40, 112, 112]           1,080
       BatchNorm2d-2         [-1, 40, 112, 112]              80
              SiLU-3         [-1, 40, 112, 112]               0
            Conv2d-4         [-1, 40, 112, 112]             360
       BatchNorm2d-5         [-1, 40, 112, 112]              80
              SiLU-6         [-1, 40, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 40, 1, 1]               0
            Conv2d-8             [-1, 10, 1, 1]             410
              SiLU-9             [-1, 10, 1, 1]               0
           Conv2d-10             [-1, 40, 1, 1]             440
          Sigmoid-11             [-1, 40, 1, 1]               0
SqueezeExcitation-12         [-1, 40, 112, 112]               0
           Conv2d-13         [-1, 24, 112, 112]             960
      BatchNorm2d-14         [-1, 24, 1

In [11]:
import os
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch

class DS_01(Dataset):
    def __init__(self, root_dir):
        self.desc = "resize[224, 224] -> transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])"
        self.root_dir = root_dir
        
        # Define default transformations (resize, to tensor, and normalization)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize to 224x224 pixels
            transforms.ToTensor(),  # Convert image to tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
        ])
        
        # Get all class names (subfolder names)
        self.class_names = sorted(os.listdir(root_dir))
        
        # Create a mapping from class name to class index
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.class_names)}
        
        # Collect all image paths and their corresponding class labels
        self.image_paths = []
        self.labels = []
        
        for class_name in self.class_names:
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                for filename in os.listdir(class_dir):
                    if filename.endswith(('.png', '.jpg', '.jpeg')):  # You can adjust extensions
                        self.image_paths.append(os.path.join(class_dir, filename))
                        self.labels.append(self.class_to_idx[class_name])
    def collate_fn(batch):
        inputs, labels = zip(*batch)  # Unzip the batch into inputs and labels
        inputs = torch.stack(inputs, dim=0)  # Stack inputs to create a tensor
        labels = torch.tensor(labels, dtype=torch.long)  # Convert labels to tensor of type long (integer)
        def one_hot_encode(labels, num_classes=74):
            # Convert integer labels to one-hot encoded labels
            return torch.eye(num_classes)[labels]
        # Apply one-hot encoding to the labels
        labels = one_hot_encode(labels)
        
        return inputs, labels
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")  # Convert to RGB to ensure consistent color channels
        
        # Get the label for the image
        label = self.labels[idx]
        
        # Apply transformations if provided
        img = self.transform(img)
        
        return img, label


# Example usage of the CustomImageDataset class

# Initialize the dataset
dataset = DS_01(root_dir='../DataSets/AirCrafts_2/Training/')

# # Example: Accessing the first image and label
# img, label = dataset[0]

# print(f"Image size: {img.size()}")
# print(f"Label: {label}")

# Optional: Create a DataLoader to load the dataset in batches
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example: Iterating over the dataloader
for images, labels in dataloader:
    print(images.shape)  # Image batch size: [32, 3, 224, 224]
    print(labels.shape)  # Label batch size: [32]
    break


torch.Size([32, 3, 224, 224])
torch.Size([32])


In [12]:
dataset.desc

'resize[224, 224] -> transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])'

In [13]:
P = test_mods(model=model,dataset=DS_01,
              train_data_src="../DataSets/AirCrafts-Copy_trn/",
              valid_data_src="../DataSets/AirCrafts-Copy_vld/",
              train_batch_size=2,
              valid_batch_size=2,
              prepare=True)
P.train(num_epochs=2)

Configuration file is saved at internal/Test/test_c.json
History will be saved at internal/Test/test_h.csv
Weights will be saved at internal/Test/test_w.pth
Data loaders are successfully created


Epoch 1/2: 100%|█| 364/364 [00:39<00:00,  9.29it/s, accuracy=0.0179, loss


Epoch 1, Train Loss: 4.4727, Train Accuracy: 0.02, Val Loss: 6.6808, Val Accuracy: 0.05
Best Model Weights Updated: Epoch 1 - Val Loss: 6.680778130070194


Epoch 2/2: 100%|█| 364/364 [00:16<00:00, 21.52it/s, accuracy=0.0604, loss


Epoch 2, Train Loss: 4.0723, Train Accuracy: 0.06, Val Loss: 4.3245, Val Accuracy: 0.11
Best Model Weights Updated: Epoch 2 - Val Loss: 4.324488139742023
Finished Training


In [14]:
from PyTorchLabFlow import set_default_config
set_default_config({
    "accuracy_loc": "Libs.accuracies.MultiAcc",
    "loss_loc": "Libs.losses.CrossEn",
    "optimizer_loc": "Libs.optimizers.OptAdamax",
    "train_data_src": "../DataSets/AirCrafts_2/Training/",
    "valid_data_src": "../DataSets/AirCrafts_2/Test/",
    "train_batch_size": 32,
    "valid_batch_size": 32
})

In [1]:
import torch
import torchvision.models as models

# Load the pre-trained VGG19 model
vgg19 = models.vgg19(weights='VGG19_Weights.DEFAULT')

# Step 2: Save the model weights (state_dict) to a file
torch.save(vgg19.state_dict(), "preTrained/vgg19.pth")

print("vgg19 weights saved successfully.")

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\LENOVO/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
 21%|██████▊                         | 118M/548M [16:28<1:00:14, 125kB/s]


RuntimeError: invalid hash value (expected "dcbb9e9d", got "25e70540fc61802670dcde10255abaf4b7a02b3802e995f61d649e31f9f6db63")

In [3]:
import torch
import torchvision.models as models

# Step 1: Load the pre-trained EfficientNet-B3 model
efficientnet_b3 = models.efficientnet_b3(pretrained=True)

# Step 2: Save the model weights (state_dict) to a file
torch.save(efficientnet_b3.state_dict(), "efficientnet_b3_weights.pth")

print("EfficientNet-B3 weights saved successfully.")

Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to C:\Users\LENOVO/.cache\torch\hub\checkpoints\efficientnet_b3_rwightman-b3899882.pth
100%|██████████████████████████████████████| 47.2M/47.2M [02:10<00:00, 380kB/s]


EfficientNet-B3 weights saved successfully.
